In [ ]:
pip install gensim

In [ ]:
pip install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#GloVe word embeddings is loaded from Google Drive
def load_glove_embeddings_from_drive(file_path):
    embeddings_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

#Main function
def main():
    glove_embeddings_path = '/content/drive/MyDrive/glove.6B.200d.txt'
    glove_embeddings = load_glove_embeddings_from_drive(glove_embeddings_path)


**GloVe Embeddings Using Siamese Neural Networks**



In [ ]:
from keras.layers import Input, Dense, Dropout, Concatenate
from keras.models import Model

from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from datasets import load_dataset

def load_glove_embeddings_from_drive(file_path):
    embeddings_index = {}
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

#Load dataset from Hugging Face
def load_dataset_from_huggingface():
    dataset = load_dataset("PiC/phrase_similarity")
    return pd.DataFrame(dataset['test'])

#Function to convert a phrase to its vector representation
def phrase_to_vector(phrase, word_embeddings):
    words = phrase.split()
    vector = np.zeros_like(word_embeddings[next(iter(word_embeddings))])
    for word in words:
        if word in word_embeddings:
            vector += word_embeddings[word]
    return vector / len(words)

#Prepare data
def prepare_data(dataset, word_embeddings):
    phrase1 = dataset['phrase1']
    phrase2 = dataset['phrase2']
    similarity_score = dataset['label']

    X1 = np.array([phrase_to_vector(phrase, word_embeddings) for phrase in phrase1])
    X2 = np.array([phrase_to_vector(phrase, word_embeddings) for phrase in phrase2])
    y = similarity_score.values
    return X1, X2, y

#Defining neural network model
def create_model(input_dim):
    input1 = Input(shape=(input_dim,))
    input2 = Input(shape=(input_dim,))

    #Defining individual branches for each input
    branch1 = Dense(128, activation='relu')(input1)
    branch2 = Dense(128, activation='relu')(input2)

    #Concatenate the outputs of the two branches
    merged = Concatenate()([branch1, branch2])

    #Additional layers
    merged = Dropout(0.2)(merged)
    merged = Dense(64, activation='relu')(merged)
    output = Dense(1, activation='sigmoid')(merged)

    #Defining the model
    model = Model(inputs=[input1, input2], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

#Main function
def main():
    #Loading GloVe embeddings from Google Drive
    glove_embeddings_path = '/content/drive/MyDrive/glove.6B.200d.txt'
    glove_embeddings = load_glove_embeddings_from_drive(glove_embeddings_path)

    #Loading dataset from Hugging Face
    dataset = load_dataset_from_huggingface()

    #Preparing data
    X1, X2, y = prepare_data(dataset, glove_embeddings)

    #Train-test split
    X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

    #Define model
    input_dim = X1_train.shape[1]
    model = create_model(input_dim)

    #Training model
    model.fit(x=[X1_train, X2_train], y=y_train, epochs=10, batch_size=32, validation_split=0.1)

    #Evaluating model
    y_pred = model.predict([X1_test, X2_test])
    y_pred_binary = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred_binary)
    print("Accuracy:", accuracy)

if __name__ == "__main__":
    main()

Epoch 1/10
45/45 [==============================] - 2s 9ms/step - loss: 0.7096 - accuracy: 0.5097 - val_loss: 0.6878 - val_accuracy: 0.5688
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6693 - accuracy: 0.5958 - val_loss: 0.7054 - val_accuracy: 0.4812
Epoch 3/10
45/45 [==============================] - 0s 4ms/step - loss: 0.6445 - accuracy: 0.6361 - val_loss: 0.6933 - val_accuracy: 0.5437
Epoch 4/10
45/45 [==============================] - 0s 3ms/step - loss: 0.6070 - accuracy: 0.6944 - val_loss: 0.6946 - val_accuracy: 0.5437
Epoch 5/10
45/45 [==============================] - 0s 3ms/step - loss: 0.5568 - accuracy: 0.7194 - val_loss: 0.7733 - val_accuracy: 0.4812
Epoch 6/10
45/45 [==============================] - 0s 3ms/step - loss: 0.5078 - accuracy: 0.7688 - val_loss: 0.7903 - val_accuracy: 0.5625
Epoch 7/10
45/45 [==============================] - 0s 4ms/step - loss: 0.4548 - accuracy: 0.8000 - val_loss: 0.7875 - val_accuracy: 0.5562
Epoch 8/10
45/45 [==

**LSTM**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import load_dataset
from keras.layers import Input, LSTM, Dense, Lambda
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K

#Load dataset from Hugging Face
def load_dataset_from_huggingface():
    dataset = load_dataset("PiC/phrase_similarity")
    return pd.DataFrame(dataset['test'])

def siamese_network_model(input_dim, max_len):
    input_1 = Input(shape=(max_len,))
    input_2 = Input(shape=(max_len,))

    #Shared LSTM layer
    shared_lstm = LSTM(128)

    #Output embeddings
    output_1 = shared_lstm(input_1[:, :, None])
    output_2 = shared_lstm(input_2[:, :, None])

    #Euclidean distance layer
    distance = Lambda(lambda x: K.abs(x[0] - x[1]))([output_1, output_2])

    #Final prediction layer
    output = Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


#Tokenize and pad sequences
def tokenize_and_pad(texts, max_len):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer.word_index

#Main function
def main():
    # Load dataset
    dataset = load_dataset_from_huggingface()

    #Prepare data
    X = dataset[['phrase1', 'phrase2']]
    y = dataset['label']

    #Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #Tokenize and pad sequences for Siamese Network
    max_len = max(max(len(phrase.split()) for phrase in X_train['phrase1']),
                  max(len(phrase.split()) for phrase in X_train['phrase2']))
    X_train_seq_1, word_index = tokenize_and_pad(X_train['phrase1'], max_len)
    X_train_seq_2, _ = tokenize_and_pad(X_train['phrase2'], max_len)
    X_test_seq_1, _ = tokenize_and_pad(X_test['phrase1'], max_len)
    X_test_seq_2, _ = tokenize_and_pad(X_test['phrase2'], max_len)

    #Concatenate sequences
    X_train_seq = np.concatenate((X_train_seq_1, X_train_seq_2), axis=1)
    X_test_seq = np.concatenate((X_test_seq_1, X_test_seq_2), axis=1)

    #Siamese Network (LSTM) Model
    model = siamese_network_model(X_train_seq.shape[1], max_len)

    #Train model
    model.fit([X_train_seq[:, :max_len], X_train_seq[:, max_len:]], y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

    #Evaluate model
    y_pred = (model.predict([X_test_seq[:, :max_len], X_test_seq[:, max_len:]]) > 0.5).astype(int)
    accuracy_siamese = accuracy_score(y_test, y_pred)
    report_siamese = classification_report(y_test, y_pred)

    print("Siamese Network (LSTM) Model:")
    print("Accuracy:", accuracy_siamese)
    print("Classification Report:")
    print(report_siamese)

if __name__ == "__main__":
    main()


Epoch 1/10
45/45 [==============================] - 8s 77ms/step - loss: 0.6950 - accuracy: 0.5056 - val_loss: 0.6905 - val_accuracy: 0.5562
Epoch 2/10
45/45 [==============================] - 1s 29ms/step - loss: 0.6933 - accuracy: 0.5035 - val_loss: 0.7085 - val_accuracy: 0.4250
Epoch 3/10
45/45 [==============================] - 2s 35ms/step - loss: 0.6911 - accuracy: 0.5312 - val_loss: 0.6966 - val_accuracy: 0.4625
Epoch 4/10
45/45 [==============================] - 2s 35ms/step - loss: 0.6898 - accuracy: 0.5243 - val_loss: 0.6991 - val_accuracy: 0.4437
Epoch 5/10
45/45 [==============================] - 2s 36ms/step - loss: 0.6893 - accuracy: 0.5410 - val_loss: 0.6921 - val_accuracy: 0.4625
Epoch 6/10
45/45 [==============================] - 1s 23ms/step - loss: 0.6916 - accuracy: 0.5194 - val_loss: 0.6826 - val_accuracy: 0.5375
Epoch 7/10
45/45 [==============================] - 1s 18ms/step - loss: 0.6895 - accuracy: 0.5160 - val_loss: 0.6959 - val_accuracy: 0.4437
Epoch 8/10
45

**TF-IDF Representation with Random Forest Classifier**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from keras.layers import Input, Embedding, LSTM, Dense, Lambda, Conv1D, GlobalMaxPooling1D, Dropout, Concatenate
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from keras import backend as K

#Load dataset from Hugging Face
def load_dataset_from_huggingface():
    dataset = load_dataset("PiC/phrase_similarity")
    return pd.DataFrame(dataset['test'])

#TF-IDF Representation with Random Forest Classifier
def tfidf_rf_model(X_train, X_test, y_train, y_test):
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)

    #Train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train_tfidf, y_train)

    #Predict and evaluate
    y_pred_rf = rf_classifier.predict(X_test_tfidf)
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    report_rf = classification_report(y_test, y_pred_rf)

    return accuracy_rf, report_rf


#Tokenize and pad sequences
def tokenize_and_pad(texts, max_len):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer.word_index

# Main function
def main():
    dataset = load_dataset_from_huggingface()

    #Prepare data
    X = dataset[['phrase1', 'phrase2']]
    y = dataset['label']

    #Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #TF-IDF Representation with Random Forest Classifier
    accuracy_rf, report_rf = tfidf_rf_model(X_train['phrase1'] + X_train['phrase2'],
                                            X_test['phrase1'] + X_test['phrase2'],
                                            y_train,
                                            y_test)

    print("TF-IDF Representation with Random Forest Classifier:")
    print("Accuracy:", accuracy_rf)
    print("Classification Report:")
    print(report_rf)

    print("\n")





TF-IDF Representation with Random Forest Classifier:
Accuracy: 0.45
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.44      0.45       204
           1       0.44      0.46      0.45       196

    accuracy                           0.45       400
   macro avg       0.45      0.45      0.45       400
weighted avg       0.45      0.45      0.45       400





ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 8)